<a href="https://colab.research.google.com/github/Alejandro280300/proyecto-IA/blob/main/04_preprocesado_y_modelo_XGBoost.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import zipfile
import pandas as pd
import io
from google.colab import files
import os

print("Sube tu archivo ZIP...")
uploaded = files.upload()

# Obtener el nombre del ZIP
zip_filename = list(uploaded.keys())[0]
print("📁 Archivo ZIP cargado:", zip_filename)

# Guardarlo temporalmente
with open(zip_filename, "wb") as f:
    f.write(uploaded[zip_filename])

# Crear carpeta donde extraer
extract_folder = "train_zip_content"
os.makedirs(extract_folder, exist_ok=True)

print("🔓 Descomprimiendo ZIP...")
with zipfile.ZipFile(zip_filename, "r") as z:
    z.extractall(extract_folder)

print("✔ ZIP descomprimido.")

# Buscar archivos CSV dentro del ZIP
csv_files = [f for f in os.listdir(extract_folder) if f.endswith(".csv")]

if not csv_files:
    raise ValueError("❌ No se encontró ningún archivo CSV dentro del ZIP.")

print("📄 Archivos CSV encontrados:", csv_files)

# Cargar el primer CSV encontrado
csv_path = os.path.join(extract_folder, csv_files[0])
print("📥 Cargando:", csv_path)

# Intentar diversas codificaciones
encodings = ["utf-8", "latin-1", "ISO-8859-1", "cp1252"]
df = None

for enc in encodings:
    try:
        df = pd.read_csv(csv_path, encoding=enc)
        print(f"✔ CSV leído correctamente con codificación: {enc}")
        break
    except:
        print(f"❌ Falló con codificación: {enc}")

if df is None:
    raise ValueError("❌ No se pudo leer el CSV con ninguna codificación.")

print("\nColumnas del dataframe:")
print(df.columns.tolist())

df.head()



Sube tu archivo ZIP...


Saving udea-ai-4-eng-20252-pruebas-saber-pro-colombia.zip to udea-ai-4-eng-20252-pruebas-saber-pro-colombia.zip
📁 Archivo ZIP cargado: udea-ai-4-eng-20252-pruebas-saber-pro-colombia.zip
🔓 Descomprimiendo ZIP...
✔ ZIP descomprimido.
📄 Archivos CSV encontrados: ['submission_example.csv', 'test.csv', 'train.csv']
📥 Cargando: train_zip_content/submission_example.csv
✔ CSV leído correctamente con codificación: utf-8

Columnas del dataframe:
['ID', 'RENDIMIENTO_GLOBAL']


,ID,RENDIMIENTO_GLOBAL
0,550236,medio-bajo
1,98545,medio-bajo
2,499179,medio-alto
3,782980,alto
4,785185,medio-bajo


In [14]:
# Eliminar duplicados
df = df.drop_duplicates()

# Detectar columnas numéricas y categóricas
num_cols = df.select_dtypes(include=['int64', 'float64']).columns.tolist()
cat_cols = df.select_dtypes(include=['object']).columns.tolist()

print("Columnas numéricas:", num_cols)
print("Columnas categóricas:", cat_cols)

# Rellenar faltantes
df[num_cols] = df[num_cols].fillna(df[num_cols].mean())

for c in cat_cols:
    df[c] = df[c].fillna(df[c].mode()[0])

# Convertir categóricas a códigos (XGBoost friendly)
for c in cat_cols:
    df[c] = df[c].astype('category').cat.codes


Columnas numéricas: ['ID']
Columnas categóricas: []


In [20]:
TARGET = "RENDIMIENTO_GLOBAL"

In [21]:
from sklearn.model_selection import train_test_split

X = df.drop(columns=[TARGET])
y = df[TARGET]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

X_train.shape, X_test.shape


((237428, 1), (59358, 1))

In [22]:
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

model = XGBRegressor(
    n_estimators=500,
    learning_rate=0.05,
    max_depth=8,
    subsample=0.8,
    colsample_bytree=0.8,
    objective="reg:squarederror",
    tree_method="hist"
)

print("Entrenando XGBoost...")
model.fit(X_train, y_train)

# Predicción
y_pred = model.predict(X_test)

# Métricas
mae = mean_absolute_error(y_test, y_pred)
rmse = mean_squared_error(y_test, y_pred) ** 0.5
r2 = r2_score(y_test, y_pred)

print("\n========== RESULTADOS XGBOOST ==========")
print("MAE:", mae)
print("RMSE:", rmse)
print("R²:", r2)


Entrenando XGBoost...

========== RESULTADOS XGBOOST ==========
MAE: 0.9988494515419006
RMSE: 1.117660209043987
R²: -0.0012400150299072266


In [23]:
model.save_model("xgb_model.json")
print("✔ Modelo guardado como xgb_model.json")
files.download("xgb_model.json")


✔ Modelo guardado como xgb_model.json


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [24]:
model.save_model("xgb_model.model")
print("✔ Modelo guardado como xgb_model.model")
files.download("xgb_model.model")


✔ Modelo guardado como xgb_model.model


/usr/local/lib/python3.12/dist-packages/xgboost/sklearn.py:1115: UserWarning: [02:52:53] WARNING: /workspace/src/c_api/c_api.cc:1575: Saving model in the UBJSON format as default.  You can use a file extension: `json` or `ubj` to choose between formats.
  self.get_booster().save_model(fname)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>